In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json
# from model import BigramLanguageModel
# from train_model import EncodingDecoding, save_encoder
import sys
import pathlib
# parent_dir = pathlib.Path(__file__).resolve().parent.parent
# models_path = str(parent_dir / "models")
# train_path = str(parent_dir / "train")
# if models_path not in sys.path:
#     sys.path.insert(0, models_path)
# if train_path not in sys.path:
#     sys.path.insert(0, train_path)

from GPTModel import *
from train_model import ByteEncodingTokenizer


In [3]:
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Hyperparameters based on device
if device == 'cuda':
    batch_size = 64
    max_iteration = 8000
    block_size = 384
    learning_rate = 3e-4
    eval_interval = 500
    n_embed = 384
    dropout = 0.2
    n_head = 6
    n_layer = 6
else:
    batch_size = 32
    max_iteration = 5000
    block_size = 256
    learning_rate = 2e-4
    eval_interval = 300
    n_embed = 192
    dropout = 0.15
    n_head = 6
    n_layer = 4

Using device: cuda


In [4]:
file_path = "reddit_chat.jsonl"

Read the file

In [5]:
conversations = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            continue

        messages = data.get('message', [])
        if not messages:
            continue

        formatted = []
        for msg in messages:
            role = msg.get('role', '').lower()
            content = msg.get('content', '').strip()
            if not content:
                continue

            if role == 'user':
                formatted.append(f"Human: {content}")
            elif role == 'assistant':
                formatted.append(f"Assistant: {content}")

        if formatted:
            conversations.append("\n".join(formatted))
print(len(conversations))

53310


Split Train and Test

In [15]:
all_text = "\n".join(conversations)
tokenizer = ByteEncodingTokenizer(all_text)

encoded = tokenizer.encode("Hello Human")
print(encoded)

decoded = tokenizer.decode(encoded)
print(decoded)

vocab_size = tokenizer.vocab_size
print(f"Vocab size: {vocab_size}")

[41, 70, 77, 77, 80, 3, 41, 86, 78, 66, 79]
Hello Human
Vocab size: 973


Tokenizer - to encode them

In [24]:
train_text = "\n\n".join(conversations[:45000])
val_text = "\n\n".join(conversations[45000:])

# Encode using your dataset-learned tokenizer
train_encoded = tokenizer.encode(train_text)
val_encoded = tokenizer.encode(val_text)

# Convert to tensors
train_data = torch.tensor(train_encoded, dtype=torch.long)
val_data = torch.tensor(val_encoded, dtype=torch.long)

# The data will remain 1D, suitable for slicing in get_batch

print(f"Train data: {train_data.numel():,} tokens")
print(f"Val data: {val_data.numel():,} tokens")

Train data: 7,207,582 tokens
Val data: 1,389,743 tokens


Get Batch

In [25]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x.to(device), y.to(device)

Fine-tune the model

In [26]:
@torch.no_grad()
def estimate_loss(eval_iters=200):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, targets=Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [28]:
print(f"\nInitializing model...")
model = GPTModelStyle(
    vocab_size=vocab_size,
    n_embed=n_embed,
    block_size=block_size,
    n_head=n_head,
    n_layer=n_layer,
    dropout=dropout,
    device=device
)
model.to(device)
print(f"Model has {sum(p.numel() for p in model.parameters()):,} parameters")


Initializing model...
Model has 11,536,333 parameters


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"\nStarting training for {max_iteration} iterations...")
print(f"Block size: {block_size}, Batch size: {batch_size}, LR: {learning_rate}")
print("-" * 60)

for iteration in range(max_iteration):
    if iteration % eval_interval == 0 or iteration == max_iteration - 1:
        losses = estimate_loss()
        print(f"Step {iteration:5d}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, targets=yb)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("\nTraining complete!")


Starting training for 8000 iterations...
Block size: 384, Batch size: 64, LR: 0.0003
------------------------------------------------------------
Step     0: train loss 7.1407, val loss 7.1406


In [ ]:
model_path = "TDGPT_n.pt"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

print("\n" + "="*60)
print("Quick generation test:")
print("="*60)

model.eval()
context = torch.tensor([tokenizer.encode("Human: Hello!\nAssistant: ")], dtype=torch.long, device=device)

with torch.no_grad():
    generated = model.generate(context, max_new_tokens=100)

result = tokenizer.decode(generated[0].tolist())
print(result)